In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Mixtral-8x7B (PEFT)

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_mixtral_peft_tuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_mixtral_peft_tuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_mixtral_peft_tuning.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview
In this notebook you will learn how to fine tune Mixtral-8x7B with QLoRa.

### Objective

*   Fine tune and merge Mixtral-8x7B model with peft train docker image (maintained by Vertex AI Model Garden). Optionally run Hyperparameter tuning to find the best parameters.
*   Deploy the finetuned model with vLLM docker image on a Vertex AI Endpoint
*   Run inference on the deployed Vertex AI Endpoint

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Step 0 - Initiatialise the notebook
### Define some helper functions and variables:

0. Define some variables and APIs


In [1]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output.
# Start with gs:// prefix, e.g. gs://foo_bucket.
BUCKET_URI = "gs://"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID
! gcloud services enable language.googleapis.com

import os

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
EXPERIMENT_BUCKET = os.path.join(BUCKET_URI, "peft")
DATA_BUCKET = os.path.join(EXPERIMENT_BUCKET, "data")
MODEL_BUCKET = os.path.join(EXPERIMENT_BUCKET, "model")

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

### Installation: *Vertex AI API*


In [2]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### Define *constants*

In [3]:
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20240112_0916_RC00"
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-train:20240126_0936_RC00"
VLLM_GPTQ_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:gptq"

### Define common functions

1. Define model deployment functions

In [4]:
from datetime import datetime
from typing import Tuple


def create_name_with_datetime(prefix: str) -> str:
    """Creates a name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model_vllm(
    model_name: str,
    model_id: str,
    service_account: str,
    machine_type: str = "n1-standard-8",
    accelerator_type: str = "NVIDIA_TESLA_V100",
    accelerator_count: int = 1,
    quantization_method: str = "",
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys trained models with vLLM into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        "--gpu-memory-utilization=0.9",
        "--max-num-batched-tokens=4096",
        "--disable-log-stats",
    ]
    if quantization_method:
        vllm_args.append(f"--quantization={quantization_method}")
    if quantization_method == "gptq":
        vllm_docker_uri = VLLM_GPTQ_DOCKER_URI
    else:
        vllm_docker_uri = VLLM_DOCKER_URI

    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=vllm_docker_uri,
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint

## Step 1 - Fine tune Mixtral-8x7B model with PEFT
This section demonstrates how to finetune the Mixtral-8x7B model, merge the finetuned LoRA adapter with the base model on Vertex AI.

Set the base model id.

In [5]:
base_model_id = "mistralai/Mixtral-8x7B"  # @param {type:"string"}

### Finetune

Use the Vertex AI SDK to create and run the custom training jobs with Vertex AI Model Garden training images.

This example uses the dataset fredmo/vertexai-qna-500 , a small dataset containing questions and answers about GCP Vertex AI Documentation. You can either use a dataset from huggingface or a custom JSONL dataset in Vertex AI text model dataset format stored in Cloud Storage. The template parameter is optional.

In order to make the finetuning efficient, you enabled quantization for loading pretrained models for finetuning LoRA models. Precision options include "4bit", "8bit", "float16" (default) and "float32", and the precision can be set via "--precision_mode".

In this section, the finetuned LoRA adapter will be saved to a GCS bucket specified by the variable lora_adapter_dir below; and you merge the LoRa adapter with the base model, and save it to a separate GCS bucket specified by merged_model_output_dir below.


### Finetune with a custom dataset

To use a custom dataset, you should supply a gs:// URI to a JSONL file in Vertex text model dataset format in the dataset_name below.

For example, you can download the template file here https://github.com/thomaslemoullec/QLora_vLLM_Mistral7B/blob/main/vertexAI_q%26a_template.json and upload it to your bucket, then reference the gs:// URI.

{"input_text":"TRANSCRIPT: \nREASON FOR EVALUATION:,\n\n LABEL:","output_text":"Chiropractic"}

To use this sample dataset that contains input_text and output_text fields, set dataset_name to gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl and template to vertex_sample. For advanced usage with custom datatset fields, see the template example and supply your own JSON template as gs:// URIs.

In [7]:
# Huggingface dataset name or gs:// URI to a custom JSONL dataset.
dataset_name = "fredmo/vertexai-qna-500"  # @param {type:"string"}
# Optional. Template name or gs:// URI to a custom template.
template = "vertex_sample"  # @param {type:"string"}
# Runs 10 training steps as a minimal example.
max_steps = 10  # @param {type:"integer"}

finetuning_precision_mode = "4bit"  # @param {type:"string"}

# Worker pool spec for 4bit finetuning.
# Finetunes Mixtral-8x7B with 4 V100 (16G).
machine_type = "n1-highmem-32"
accelerator_type = "NVIDIA_TESLA_V100"
accelerator_count = 4

# Finetunes Mixtral-8x7B with 2 L4 (24G).
# machine_type = "g2-standard-24"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 2

replica_count = 1


# Setup training job.
job_name = create_name_with_datetime("mixtral-lora-train")
train_job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
)

# Create a GCS folder to store the LORA adapter.
lora_adapter_dir = create_name_with_datetime("mixtral-lora-adapter")
lora_output_dir = os.path.join(MODEL_BUCKET, lora_adapter_dir)

# Create a GCS folder to store the merged model with the base model and the
# finetuned LORA adapter.
merged_model_dir = create_name_with_datetime("mixtral-merged-model")
merged_model_output_dir = os.path.join(MODEL_BUCKET, merged_model_dir)

# Pass training arguments and launch job.
train_job.run(
    args=[
        "--task=causal-language-modeling-lora",
        f"--pretrained_model_id={base_model_id}",
        f"--dataset_name={dataset_name}",
        f"--output_dir={lora_output_dir}",
        f"--merge_base_and_lora_output_dir={merged_model_output_dir}",
        "--lora_rank=16",
        "--lora_alpha=32",
        "--lora_dropout=0.05",
        "--warmup_steps=10",
        f"--max_steps={max_steps}",
        "--learning_rate=2e-4",
        f"--precision_mode={finetuning_precision_mode}",
        f"--template={template}",
    ],
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    boot_disk_size_gb=500,
)

print("The finetuned Lora adapter can be found at: ", lora_output_dir)
print(
    "The finetuned Lora adapter merged with the base model can be found at: ",
    merged_model_output_dir,
)

## Step 2 - Deploy the finetuned model with vLLM docker image

This section uploads the model to Model Registry and deploys it on the Endpoint.
The model deployment step will take ~15 minutes to complete.


NOTE: vLLM requires a merged model with the base model and the finetuned LoRA adapter. Based on your business need, if you need the base model and the finetuned LoRA weight to be served separately, please consider using the regular Vertex AI serving instead.

In [11]:
# Finds Vertex AI prediction supported accelerators and regions in
# https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.

# Sets L4 to deploy Mixtral-8x7B.
machine_type = "g2-standard-96"
accelerator_type = "NVIDIA_L4"
accelerator_count = 8

model_with_peft, endpoint_with_peft = deploy_model_vllm(
    model_name=create_name_with_datetime(prefix="mixtral-peft-serve-vllm"),
    model_id=merged_model_output_dir,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
)

print("endpoint_name:", endpoint_with_peft.name)

## Step 3 Run inference to evaluate the finetuned model (vLLM Vertex endpoint)


In [ ]:
instance = {
    "prompt": "What is Model Garden?",
    "n": 1,
    "max_tokens": 250,
    "temperature": 1.0,
    "top_p": 1.0,
    "top_k": 10,
}
response = endpoint_with_peft.predict(instances=[instance])
print(response.predictions[0])

## Clean up resources

In [ ]:
# Undeploy models and delete endpoints.
endpoint_with_peft.delete(force=True)

# Delete models.
model_with_peft.delete()